<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/src/experimental-notebooks/emotion_classifier_FT_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install -q --no-cache transformers sentencepiece datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 113.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 271.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 210.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 233.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 311.6 MB/s eta 0:00:00


In [30]:
import transformers
from transformers import AutoTokenizer, Trainer, AutoModelForSequenceClassification
import torch
from torch import nn
import pandas as pd
from datasets import load_dataset

In [5]:
dataset = load_dataset("emotion")
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [6]:
dataset['train'][5]

{'text': 'ive been feeling a little burdened lately wasnt sure why that was',
 'label': 0}

In [7]:
df = dataset['train'].to_pandas()
df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [8]:
features = dataset['train'].features
features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [9]:
features['label'].int2str(0)

'sadness'

In [32]:
id2label = {
    idx: features['label'].int2str(idx) for idx in range(6)
}

label2id = {
    v: k for k, v in id2label.items()
}

id2label, label2id

({0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'},
 {'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5})

In [11]:
df['label'].value_counts(normalize=True).sort_index()

0    0.291625
1    0.335125
2    0.081500
3    0.134937
4    0.121063
5    0.035750
Name: label, dtype: float64

In [12]:
checkpoint = "microsoft/MiniLM-L12-H384-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [15]:
dataset['train']['text'][:2]

['i didnt feel humiliated',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake']

In [16]:
tokenizer(dataset['train']['text'][:2])

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102], [101, 1045, 2064, 2175, 2013, 3110, 2061, 20625, 2000, 2061, 9636, 17772, 2074, 2013, 2108, 2105, 2619, 2040, 14977, 1998, 2003, 8300, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [19]:
def tokenizeText(examples):
    return tokenizer(examples['text'], truncation=True, max_length=512)

Use the `map()` method of the dataset to apply a pre-defined function on your dataset.

In [21]:
dataset = dataset.map(tokenizeText, batched=True)
dataset

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

Boost the signal of a rarer class with a higher class weight than that of a more common class. We do this since the data is imabalanced and we don't want the data points with the rarer classes to be overlooked by the model.

In [23]:
classWeights = (1 - (df['label'].value_counts().sort_index() / len(df))).values
classWeights

array([0.708375 , 0.664875 , 0.9185   , 0.8650625, 0.8789375, 0.96425  ])

In [26]:
classWeights = torch.from_numpy(classWeights).float()
# classWeights = torch.from_numpy(classWeights).float().to('cuda')
classWeights

tensor([0.7084, 0.6649, 0.9185, 0.8651, 0.8789, 0.9643])

Have the label column correspond to the trainer's expectations. Rename it to labels.

In [27]:
dataset = dataset.rename_column('label', 'labels')

Create a custom trainer to sub-class any of the arguments you have for the trainer. If we want the `compute_loss()` function, we create a custom sub-class of the trainer.

In [29]:
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)

        # Get the output classes as logits to ultimately compare with the testing classes in the dataset
        logits = outputs.get('logits')
        labels = inputs.get('labels')

        lossFunction = nn.CrossEntropyLoss(weight=classWeights)

        loss = lossFunction(logits, labels)

        return (loss, outputs) if return_outputs else loss

In [31]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=6,
    id2label=id2label,
    label2id=label2id
)

NameError: ignored